### Problem 1

(100 points) Consider the following problem. 

$$
\hbox{min } f =  x_1^2+(x_2-3)^2
$$
$$
\hbox{s.t. } g_1 =  x_2^2-2x_1\leq 0
$$
$$
 g_2= (x_2-1)^2+5x_1-15\leq 0
$$

Implement an SQP algorithm with line search to solve this problem, starting from
${\bf x}_0=(1,1)^T$. Incorporate the QP subproblem. Use BFGS
approximation for the Hessian of the Lagrangian. Use the
merit function and Armijo Line Search to find the step size.

In [1]:
##
# imports

import numpy as np
import scipy.optimize as opt

In [2]:
##
# function defs

def cost_f(x):
    return x[0]**2.0 + (x[1] - 3)**2.0

def g_1(x):
    return x[1]**2.0 - 2.0 * x[0]

def g_2(x):
    return (x[1] - 1)**2.0 + 5.0 * x[0] - 15.0

def g_f(x):
    return [g_1(x), g_2(x)]

def grad_cost_f(x):
    return np.array([2.0 * x[0], 2.0 * x[1] - 6.0])

def grad_g_f(x):
    return np.array([[-2.0, 5.0], [2.0 * x[1], 2.0 * x[1] - 2.0]])

def merit_f(x, mu, w_j):
    g = grad_g_f(x)
    return cost_f(x) + np.transpose(g) @ w_j

def del_merit_del_a_f(x, mu, w_j, s):
    dgda = []
    grad_g = grad_g_f(x)
    print(grad_g)
    for i in range(len(x)):
        if 0 < (grad_g[:,i] @ s):
            dgda.append(np.transpose(grad_g[:,i]) @ s)
        else:
            dgda.append(0.0)
    dgda = np.array(dgda)
    return np.transpose(grad_cost_f(x)) @ s + np.transpose(dgda) @ w_j

def line_search_phi(x, a, t, w_j, s, mu):
    print(merit_f(x, mu, w_j))
    del_merit_del_a_f(x, mu, w_j, s)
    return merit_f(x, mu, w_j) + t * a * del_merit_del_a_f(x, mu, w_j, s)

def line_search(x, t, s, mu):
    return 0.5
    w_j_k = np.abs(mu)
    i = 0
    a = 1
    while line_search_phi(x, a, t, w_j_k, s, mu) < cost_f(x - a * grad_cost_f(x)):
        a = 0.5 * a
        i = i + 1
        if (i > 100):
            print("line search did not converge.")
            return a
    return a

def QP(x, lam, W):
    max_iter_count = 10000
    for i in range(max_iter_count):
        A = grad_g_f(x)
        fx = grad_cost_f(x)
        h = g_f(x)
        top_big_mat = np.hstack(tuple([W, np.transpose(A)]))
        zero_mat = np.zeros(shape=np.shape(A))
        bot_big_mat = np.hstack(tuple([A, zero_mat]))
        big_mat = np.vstack(tuple([top_big_mat, bot_big_mat]))
        #rint(big_mat)
        try:
            s_lam = np.linalg.inv(big_mat) @ (-1.0 * np.hstack(tuple([fx, h])))
        except:
            print(big_mat)
        s = s_lam[0:2]
        lam = s_lam[2:4]
        mu_mag = np.linalg.norm(lam) # mu and lam are interchangeable here, no equality constraints
        if mu_mag > 0:
            return s, lam
        elif mu_mag <= 0: # add more here
            pass
    print("QP hit max iteration count!")
    return [s, lam]

def BFGS(x_k, x_kp1, H_k, s):
    y = grad_cost_f(x_kp1) - grad_cost_f(x_k)
    mid_term = (y @ np.transpose(y)) / (np.transpose(y) @ s)
    last_term = ((H_k @ s) @ (np.transpose(s) @ H_k) / ((np.transpose(s) @ H_k) @ s))
    return H_k + mid_term - last_term

def SQP(x_0, lam_0, mu_0, H_0, epi):
    t = 0.1
    H_k = H_0
    mu = mu_0
    lam = lam_0
    x_k = x_0
    div_L = cost_f(x_k) + np.transpose(mu) @ grad_g_f(x_k)
    norm_div_L = np.linalg.norm(div_L)
    counter = 0
    while epi < norm_div_L:
        W_k = H_k[0:2, 0:2]
        (s_k, mu) = QP(x_k, mu, W_k)
        alpha_k = line_search(x_k, t, s_k, mu)
        x_kp1 = x_k + alpha_k * s_k
        H_est_kp1 = BFGS(x_k, x_kp1, H_k, s_k)
        div_L = cost_f(x_kp1) + np.transpose(x_kp1) @ grad_g_f(x_kp1)
        norm_div_L = np.linalg.norm(div_L)
        x_k = x_kp1
        H_k = H_est_kp1
        counter = counter + 1
        if counter > 1000:
            print(norm_div_L)
            print("Did not converge in SQP!")
            return x_k
    return x_k


In [3]:
##
# checking real answer 
x_0 = np.array([1.0, 1.0])
constraints = opt.NonlinearConstraint(g_f, [-np.inf, -np.inf], [0.0, 0.0])
result = opt.minimize(cost_f, x_0, constraints=constraints)
print(result.message)
print(result.x)

Optimization terminated successfully
[1.06020715 1.45616424]


In [4]:
##
# running this version

H_0 = np.eye(2)
mu_0 = lam_0 = np.array([0.0, 0.0])
epi = 1e-3
x_ans = SQP(x_0, lam_0, mu_0, H_0, epi)
print("x_ans: ", x_ans)

<ipython-input-2-d232aaa97793>:83: RuntimeWarning: invalid value encountered in double_scalars
  mid_term = (y @ np.transpose(y)) / (np.transpose(y) @ s)


QP hit max iteration count!
x_ans:  [nan nan]


* DONE
    * Set up basic structure
    * most of QP
    * checked what answer I should be getting.
    * wrote merit function line search
    * wrote BFGS function

* TODO 
    * debug